In [152]:
import pandas as pd
import requests
from io import StringIO

In [153]:
response = requests.get('https://dl.dropbox.com/s/06ni77wvrhukth3/scimagojr_full_2020-03-31.csv')
content = StringIO(response.content.decode("utf8"))
df = pd.read_table(content, sep=";")
df['Quartile'] = df.apply(lambda row: 4 if row['SJR Quartile'] == '-' else int(row['SJR Quartile'][1]), axis = 1)

df.drop(['Unnamed: 0', 'Rank', 'Sourceid', 'Issn', 'SJR', 'SJR Quartile',
 'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
 'Total Cites (3years)', 'Citable Docs. (3years)', 'Cites / Doc. (2years)',
 'Ref. / Doc.', 'Country', 'Publisher', 'Coverage', 'Categories'], axis=1, inplace=True)

In [154]:
print(df.columns.values)
df.sample(10)

['Title' 'Type' 'Area' 'Quartile']


,Title,Type,Area,Quartile
16787,CALL-EJ,journal,Computer Science Applications,4
45892,Erga-Logoi,journal,Literature and Literary Theory,4
33095,Estudos Feministas,journal,Gender Studies,3
12051,Psicologia Clinica dello Sviluppo,journal,Clinical Psychology,4
21425,Settler Colonial Studies,journal,Demography,2
13664,SKY 2016 - 7th International Workshop on Softw...,conference and proceedings,Computer Graphics and Computer-Aided Design,4
61620,Mycobiology,journal,Plant Science,2
28870,African Journal of Emergency Medicine,journal,Emergency Medicine,2
56516,Revista Brasileira de Ginecologia e Obstetricia,journal,Obstetrics and Gynecology,3
60175,Culture and Religion,journal,Philosophy,2


In [155]:
# df_cs = df.loc[df['Area'].str.contains('computer science', case = False, regex = False) & df['SJR Quartile'].str.contains('Q1|Q2')]
df_cs = df.loc[df['Area'].str.contains('Computer Science', case = False, regex = False)]
df_cs['Area'] = 'Computer Science'
# df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
df_ling = df.loc[df['Area'].str.contains('Linguistics', case = False, regex = False)]
df_ling['Area'] = 'Linguistics'

/home/jupyter_notebook/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyter_notebook/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [156]:
ling_cs = pd.merge(df_cs, df_ling, how='inner', on=['Title', 'Type'],  suffixes=('_cs', '_ling'))
ling_cs = ling_cs.groupby('Title').agg({'Quartile_ling': ['min', 'max'], 'Quartile_cs': ['min', 'max']})
ling_cs


Quartile_ling      \
                                                             min max   
Title                                                                  
2009 IEEE International Technology Management C...             4   4   
2015 5th International Conference on Digital In...             4   4   
2015 IEEE 7th International Workshop on Managin...             4   4   
2015 IEEE Conference on Wireless Sensors, ICWiS...             4   4   
2015 IRCOBI Conference Proceedings - Internatio...             4   4   
2015 International Symposium on Intelligent Sig...             4   4   
ACM International Conference Proceeding Series                 4   4   
Applied Ontology                                               1   1   
Argument and Computation                                       1   1   
CALL-EJ                                                        2   2   
Computational Linguistics                                      1   1   
Computational Linguistics in the Netherlands Jo...             4   4   
Computer Assisted Language Learning                            1   1   
Computers and Composition                                      1   1   
Digital Scholarship in the Humanities                          2   2   
ICCSE 2016 - 11th International Conference on C...             4   4   
International Journal of Computer-Assisted Lang...             2   2   
JALT CALL Journal                                              2   2   
Journal of Logic, Language and Information                     2   2   
Journal of Visual Languages and Computing                      2   2   
Komp'juternaja Lingvistika i Intellektual'nye T...             4   4   
Language Documentation and Conservation                        1   1   
Language Learning and Technology                               1   1   
Perspective Technologies and Methods in MEMS De...             4   4   
Pragmatics and Cognition                                       2   2   
Proceedings of 2016 8th International Conferenc...             4   4   
Procesamiento de Lenguaje Natural                              2   2   
ReCALL                                                         1   1   
Speech Communication                                           1   1   
Synthesis Lectures on Human Language Technologies              1   1   
TAL Traitement Automatique des Langues                         3   3   
Texto Livre                                                    4   4   

                                                   Quartile_cs      
                                                           min max  
Title                                                               
2009 IEEE International Technology Management C...           4   4  
2015 5th International Conference on Digital In...           4   4  
2015 IEEE 7th International Workshop on Managin...           4   4  
2015 IEEE Conference on Wireless Sensors, ICWiS...           4   4  
2015 IRCOBI Conference Proceedings - Internatio...           4   4  
2015 International Symposium on Intelligent Sig...           4   4  
ACM International Conference Proceeding Series               4   4  
Applied Ontology                                             1   1  
Argument and Computation                                     2   2  
CALL-EJ                                                      4   4  
Computational Linguistics                                    2   2  
Computational Linguistics in the Netherlands Jo...           4   4  
Computer Assisted Language Learning                          1   1  
Computers and Composition                                    1   1  
Digital Scholarship in the Humanities                        3   3  
ICCSE 2016 - 11th International Conference on C...           4   4  
International Journal of Computer-Assisted Lang...           4   4  
JALT CALL Journal                                            4   4  
Journal of Logic, Language and Information                   2   2  
Journal of V

In [157]:
s1.to_csv('res.csv', sep=";")